In [1]:
import os
import pandas as pd

In [ ]:

base_dir = 'EHR'

for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    # if folder not in ['columbia', 'montefiore', 'mshs', 'nyu', 'wcm']:
    if folder not in ['nyu']:

        continue
    print(folder_path)    
    if os.path.isdir(folder_path):
        print(f"--- Processing folder: {folder} ---")
        
        for file in os.listdir(folder_path):
            if file.endswith(".csv"):  # Check if the file is a CSV
                file_path = os.path.join(folder_path, file)
                
                try:
                    df = pd.read_csv(file_path, nrows=5)  # Read only the first 5 rows
                    print(f"\nFile: {file}")
                    display(df.head(2))  # Print the first few rows
                except Exception as e:
                    display(f"Error reading file {file}: {e}")


In [ ]:


def process_all_domain(_demo, _dx, _rx, _vital, _lab, cllab_list):
    print('###### Input demo: ', _demo.shape)
    # demo_all = _demo.drop_duplicates()
    # print('Drop duplicate demo rows', _demo.shape[0]- demo_all.shape[0])
    demo_all = _demo.copy() # assume no duplicate originally
    demo_all["patid"] = demo_all["patid"].astype(int)
    demo_all["sex"] = demo_all["sex"].astype(str)
    demo_all["hispanic"] = demo_all["hispanic"].astype(str)
    demo_all["race"] = demo_all["race"].astype(str)
    demo_all["birth_date"] = pd.to_datetime(demo_all["birth_date"])
    print('Convert column type for demo_all')

    before_dropna = demo_all.shape[0]
    demo_all = demo_all.dropna(subset=["patid", "sex", "race", "birth_date"], how='any')
    print(f"--Drop rows {before_dropna - demo_all.shape[0]} with any na in columns.")
    
    before_dedup = demo_all.shape[0]
    demo_all = demo_all.drop_duplicates()
    print(f"--Final drop duplicate rows {before_dedup - demo_all.shape[0]} after processing")
    print(f"Final shape: {demo_all.shape}")
    print('----------\n')



    # Processing dx_all
    print('###### Input dx: ', _dx.shape)
    dx_all = _dx.dropna(subset=['dx'])
    print('--Drop nan [dx] rows', _dx.shape[0]- dx_all.shape[0])

    dx_all["dx_date_fill"] = dx_all["dx_date"].where(dx_all["dx_date"].notnull(), dx_all["admit_date"])
    dx_all = dx_all.drop(['admit_date', 'dx_date'], axis=1)
    print('Get dx_date_fill from admit_date and dx_date, drop dx_date and admit_date columns')
    before_dedup = dx_all.shape[0]
    dx_all = dx_all.drop_duplicates()
    print(f"--Drop duplicate {before_dedup - dx_all.shape[0]}  rows.")
    
    dx_all["patid"] = dx_all["patid"].astype(int)
    dx_all["dx"] = dx_all["dx"].astype(str)
    dx_all["dx_type"] = pd.to_numeric(dx_all["dx_type"], errors="coerce")
    dx_all["dx_date_fill"] = pd.to_datetime(dx_all["dx_date_fill"])
    print('Convert column type for dx')

    before_dropna = dx_all.shape[0]
    dx_all = dx_all.dropna(subset=["dx_type", "dx_date_fill", "patid", "dx"], how='any')
    print(f"--Drop rows {before_dropna - dx_all.shape[0]} with any na in columns.")
    
    dx_all["dx_type"] = dx_all["dx_type"].astype(int)

    before_dedup = dx_all.shape[0]
    dx_all = dx_all.drop_duplicates()
    print(f"--Final drop duplicate rows {before_dedup - dx_all.shape[0]} after processing")
    print(f"Final shape: {dx_all.shape}")
    print('----------\n')


    # Processing rx_all
    print('###### Input rx: ', _rx.shape)
    rx_all = _rx.dropna(subset=['rxnorm_cui', 'patid'], how='any')
    print('--Drop nan [rxnorm_cui] rows', _rx.shape[0]- rx_all.shape[0])
    
    rx_all["patid"] = rx_all["patid"].astype(int)
    rx_all["rx_start_date"] = pd.to_datetime(rx_all["rx_start_date"], errors="coerce")
    rx_all["rxnorm_cui"] = pd.to_numeric(rx_all["rxnorm_cui"], errors="coerce")
    print('Convert column type for rx')

    before_dropna = rx_all.shape[0]
    rx_all = rx_all.dropna(subset=["rx_start_date", "rxnorm_cui", "patid"], how='any')
    print(f"--Drop rows {before_dropna - rx_all.shape[0]} with any na in columns.")

    rx_all["rxnorm_cui"] = rx_all["rxnorm_cui"].astype(int)
    # rx_all["patid"] = rx_all["patid"].astype(int)

    before_dedup = rx_all.shape[0]
    rx_all = rx_all.drop_duplicates()
    print(f"--Final drop duplicate rows {before_dedup - rx_all.shape[0]} after processing")
    print(f"Final shape: {rx_all.shape}")
    print('----------\n')


    # Processing vital_all
    print('###### Input vital: ', _vital.shape)
    vital_all = _vital.dropna(subset=['measure_date'])
    print('--Drop nan [measure_date] rows', _vital.shape[0]- vital_all.shape[0])
       
    vital_all["patid"] = vital_all["patid"].astype(int)
    vital_all["measure_date"] = pd.to_datetime(vital_all["measure_date"], errors="coerce")
    vital_all["ht"] = pd.to_numeric(vital_all["ht"], errors="coerce")
    vital_all["wt"] = pd.to_numeric(vital_all["wt"], errors="coerce")
    vital_all["diastolic"] = pd.to_numeric(vital_all["diastolic"], errors="coerce")
    vital_all["systolic"] = pd.to_numeric(vital_all["systolic"], errors="coerce")
    print('Convert column type for vital')

    before_dropna = vital_all.shape[0]
    vital_all = vital_all.dropna(subset=["ht", "wt", "diastolic", "systolic"], how='all')
    vital_all = vital_all.dropna(subset=["patid", "measure_date"], how='any')
    print(f"--Drop rows {before_dropna - vital_all.shape[0]} with any na in columns.")
    
    # vital_all["patid"] = vital_all["patid"].astype(int)

    before_dedup = vital_all.shape[0]
    vital_all = vital_all.drop_duplicates()
    print(f"--Final drop duplicate rows {before_dedup - vital_all.shape[0]} after processing")
    print(f"Final shape: {vital_all.shape}")
    print('----------\n')


    # Processing lab_all
    print('###### Input lab: ', _lab.shape)
    # display(_lab)
    lab_all = _lab.dropna(subset=['result_num','lab_loinc' ], how='any')
    print('--Drop nan [result_num, lab_loinc] rows', _lab.shape[0]- lab_all.shape[0])
    # display(lab_all)

    lab_all['lab_loinc'] = lab_all['lab_loinc'].astype(str)
    lab_all = lab_all[lab_all.lab_loinc.isin(cllab_list)]
    print(f"--Filtered to {lab_all.shape[0]} rows using cllab_list.")
    # display(lab_all)

    lab_all['patid'] = lab_all['patid'].astype(int)
    # lab_all['lab_loinc'] = lab_all['lab_loinc'].astype(str)
    lab_all['specimen_date'] = pd.to_datetime(lab_all['specimen_date'], errors='coerce')
    lab_all['result_num'] = pd.to_numeric(lab_all['result_num'], errors='coerce')
    lab_all['result_unit'] = lab_all['result_unit'].astype(str)
    lab_all['norm_range_low'] = pd.to_numeric(lab_all['norm_range_low'], errors='coerce')
    lab_all['norm_range_high'] = pd.to_numeric(lab_all['norm_range_high'], errors='coerce')
    print('Convert column type for lab')
    # display(lab_all)

    before_dropna = lab_all.shape[0]
    lab_all = lab_all.dropna(subset=['patid','lab_loinc', 'result_num', 'result_unit', 'specimen_date'], how='any')
    print(f"--Drop rows {before_dropna - lab_all.shape[0]} with any na in columns.")
    
    before_dedup = lab_all.shape[0]
    lab_all = lab_all.drop_duplicates()
    print(f"--Final drop duplicate rows {before_dedup - lab_all.shape[0]} after processing")
    print(f"Final shape: {lab_all.shape}")
    print('----------\n')

    return demo_all, dx_all, rx_all, vital_all, lab_all


def get_site_data(site, preview=None):
    cols = ['Class', 'Code', 'Name', 'Acceptable Sample Source', 'Source Suitable', 'Rangelow', 'Rangehigh', 'Unit of Ranges', 'Range Specs', 'Range Applicable', 'Range reference', 'Corresponding test in paper 1', 'Note']  

    cllab = pd.read_excel('Collected_lab_tests_for_ADRD_evaluation_M_online.xlsx', index_col=None, skiprows=4)  

    cllab.columns = cols
    cllab_list = cllab.Code.unique().tolist()
    print('\tcllab list has codes: ', len(cllab_list))
    
    
    democols = ['patid',	'birth_date',		'sex',	'hispanic',	'race']
    print('Loading demographics...')
    _sDemo = pd.read_csv(f'../{site}/demographic.csv', usecols=democols, nrows=preview )


    dxcols = ['patid',	'encounterid'	,	'admit_date',	'dx'	,'dx_type',	'dx_date']
    print('Loading condition...')
    _sCond = pd.read_csv(f'../{site}/diagnosis.csv', usecols=dxcols, nrows=preview )
    

    rxcols = ['patid','encounterid'	, 'rx_start_date', 'rxnorm_cui']
    print('Loading drugs...')
    _sDrug = pd.read_csv(f'../{site}/prescribing.csv', usecols=rxcols, nrows=preview )


    labcols = ['patid', 'encounterid'	, 'specimen_date', 'lab_loinc', 'result_num',  'result_unit' , 'norm_range_low', 'norm_range_high']
    print('Loading lab tests...')
    _sLab = pd.read_csv(f'../{site}/lab_result_cm.csv', usecols=labcols, nrows=preview )

    # display(_sLab)
    vitalcols = [	'patid',	'encounterid'	,'measure_date',	'ht',	'wt',	'diastolic',	'systolic']
    print('Loading vital ...')
    try:
        _sVital = pd.read_csv(f'../{site}/vital.csv', usecols=vitalcols, nrows=preview )
    except:
        _sVital = pd.DataFrame(columns=vitalcols)


    demo_all, dx_all, rx_all, vital_all, lab_all = process_all_domain(_sDemo, _sCond, _sDrug, _sVital, _sLab, cllab_list)
    print('Adding prefix to patid...')
    demo_all['patid'] = site[:2] + demo_all['patid'].astype(str)
    dx_all['patid'] = site[:2]  + dx_all['patid'].astype(str)
    rx_all['patid'] = site[:2]  + rx_all['patid'].astype(str)
    vital_all['patid'] = site[:2]  + vital_all['patid'].astype(str)
    lab_all['patid'] = site[:2]  + lab_all['patid'].astype(str)
    print('Done!')
    return demo_all, dx_all, rx_all, vital_all, lab_all




In [ ]:
wcdemo_all, wcdx_all, wcrx_all, wcvital_all, wclab_all = get_site_data('wcm')

In [7]:
import pickle
pickle.dump(wcdemo_all, open('EHR/wcdemo_all.pkl', 'wb'))
pickle.dump(wcdx_all, open('EHR/wcdx_all.pkl', 'wb'))
pickle.dump(wcrx_all, open('EHR/wcrx_all.pkl', 'wb'))
pickle.dump(wcvital_all, open('EHR/wcvital_all.pkl', 'wb'))
pickle.dump(wclab_all, open('EHR/wclab_all.pkl', 'wb'))


In [ ]:
nydemo_all, nydx_all, nyrx_all, nyvital_all, nylab_all = get_site_data('nyu')

In [10]:
pickle.dump(nydemo_all, open('EHR/nydemo_all.pkl', 'wb'))
pickle.dump(nydx_all, open('EHR/nydx_all.pkl', 'wb'))
pickle.dump(nyrx_all, open('EHR/nyrx_all.pkl', 'wb'))
pickle.dump(nyvital_all, open('EHR/nyvital_all.pkl', 'wb'))
pickle.dump(nylab_all, open('EHR/nylab_all.pkl', 'wb'))


In [ ]:
msdemo_all, msdx_all, msrx_all, msvital_all, mslab_all = get_site_data('mshs')

In [13]:
pickle.dump(msdemo_all, open('EHR/msdemo_all.pkl', 'wb'))
pickle.dump(msdx_all, open('EHR/msdx_all.pkl', 'wb'))
pickle.dump(msrx_all, open('EHR/msrx_all.pkl', 'wb'))
pickle.dump(msvital_all, open('EHR/msvital_all.pkl', 'wb'))
pickle.dump(mslab_all, open('EHR/mslab_all.pkl', 'wb'))


In [ ]:
modemo_all, modx_all, morx_all, movital_all, molab_all = get_site_data('montefiore')

In [257]:
pickle.dump(modemo_all, open('EHR/modemo_all.pkl', 'wb'))
pickle.dump(modx_all, open('EHR/modx_all.pkl', 'wb'))
pickle.dump(morx_all, open('EHR/morx_all.pkl', 'wb'))
pickle.dump(movital_all, open('EHR/movital_all.pkl', 'wb'))
pickle.dump(molab_all, open('EHR/molab_all.pkl', 'wb'))


In [ ]:
codemo_all, codx_all, corx_all, covital_all, colab_all = get_site_data('columbia')

In [259]:
pickle.dump(codemo_all, open('EHR/codemo_all.pkl', 'wb'))
pickle.dump(codx_all, open('EHR/codx_all.pkl', 'wb'))
pickle.dump(corx_all, open('EHR/corx_all.pkl', 'wb'))
pickle.dump(covital_all, open('EHR/covital_all.pkl', 'wb'))
pickle.dump(colab_all, open('EHR/colab_all.pkl', 'wb'))
